# Import necessary modules

In [2]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

import time
import random


# Read data file

In [ ]:
df = pd.read_csv(r"D:\fpt47\Downloads\car_detail_en.csv")
df.head(5)
df.shape

# Day post

In [ ]:
links = df["url"]
print(links)
links.isnull().sum()

In [ ]:
options = Options()
options.page_load_strategy = 'none'  # Set the page load strategy

driver = webdriver.Chrome(options = options)  # Pass the options to the WebDriver
with open("day_post.txt", "a", encoding='utf-8') as f:
    start = 961
    for i, link in enumerate(links[start:]):
        driver.get(link)
        current_link = driver.current_url
        if link != current_link:
            print(i + 1 + start)
            print(f"Redirect occurred!")
            f.write("Redirect occurred!" + "\n")
        else:
            wait = WebDriverWait(driver, 3)
            class_notes = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "notes")))
            print(i + 1 + start)
            print(class_notes.text)
            f.write(class_notes.text + "\n")
    driver.quit()

In [ ]:
user_data_dir = "C:/Users/fpt47/AppData/Local/Google/Chrome/User Data - Copy"
service = Service(executable_path = "D:/fpt47/Downloads/chromedriver-win64/chromedriver.exe")

options = Options()
options.page_load_strategy = 'none'
options.add_argument(f"user-data-dir={user_data_dir}")
options.add_argument("--profile-directory=Default")

driver = webdriver.Chrome(service = service, options=options)
driver.get(links[0])

try:
    # Find the iframe
    wait = WebDriverWait(driver, 5)
    iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))

    # Switch to the iframe's context
    driver.switch_to.frame(iframe)

    # Now you can find the element within the iframe
    recaptcha_button = wait.until(EC.presence_of_element_located((By.ID, "recaptcha-anchor")))

    # Interact with the element (e.g., click it)
    time.sleep(random.uniform(4, 5))
    actions = ActionChains(driver)
    actions.move_to_element(recaptcha_button).perform()
    recaptcha_button.click()
except TimeoutException:
    # If iframe is not found, handle the exception
    print("The iframe is not present on the page. Performing alternative action...")
finally:
    # Switch back to the main content after interacting with the element
    driver.switch_to.default_content()

    # # Set the page load strategy
    # user_data_dir = r"C:\Users\fpt47\AppData\Local\Google\Chrome\User Data - Copy - Copy"
    # service = Service(executable_path = "D:/fpt47/Downloads/chromedriver-win64/chromedriver.exe")

    # options = Options()
    # options.page_load_strategy = 'none'
    # options.add_argument(f"user-data-dir={user_data_dir}")
    # options.add_argument("--profile-directory=Default")

    # driver = webdriver.Chrome(service = service, options=options)
    # driver.get(links[0])

    with open("day_post.txt", "a", encoding = 'utf-8') as f:
        start = 1229
        for i, link in enumerate(links[start:]):
            driver.get(link)
            current_link = driver.current_url
            if link != current_link:
                print(i + 1 + start)
                print(f"Redirect occurred!")
                f.write("Redirect occurred!" + "\n")
            else:
                wait = WebDriverWait(driver, 3)
                class_notes = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "notes")))
                print(i + 1 + start)
                print(class_notes.text)
                f.write(class_notes.text + "\n")
        driver.quit()

In [ ]:
day_post = []
with open("day_post.txt", "r", encoding = "utf-8") as f:
    line = f.readline()
    while line:
        line = line.split()
        if line[0] == "Redirect":
            print("NULL")
            day_post.append(None)
        else:
            day = line[2]      
            print(day)
            day_post.append(day)
        line = f.readline()
df["posting_date"] = day_post

# Xử lý giá

In [64]:
import re
def text_to_number(text):
    """
    Converts text like "3 Billion 550 Million" to a numeric value.
    Handles cases with missing billions or millions.
    """
    word_to_num = {
        "Billion": 1,
        "Million": 1e-3
    }
    if pd.isna(text):
        return 0
    pattern = r"(\d+)?( Billion| Million)?"
    matches = re.findall(pattern, text)
    total = 0
    for match in matches:
        num, word = match
        if not word:
            pass
        else:
            total += float(num)*word_to_num[word.strip()]
    return round(total, 3)
df = df.rename(columns = {"price. price": "price"})
df["price_in_billion"] = df["price"].apply(text_to_number)


# Missing describe row scraping

In [ ]:
file_path = r"D:\fpt47\Downloads\miss_describe.txt"
miss_describe_row_index = []
with open(file_path, "r") as f:
    line = f.readline()
    while line:
        row_index = int(line.strip())
        miss_describe_row_index.append(row_index)
        # print(row_index)
        line = f.readline()
# print(miss_describe_row_index)
print(len(miss_describe_row_index))
for i in miss_describe_row_index:
    print(df.at[i, "describe"])


In [ ]:
options = Options()
options.page_load_strategy = 'none'  # Set the page load strategy

driver = webdriver.Chrome(options = options)  # Pass the options to the WebDriver
with open("missing_describe.txt", "a", encoding='utf-8') as f:
    for i in miss_describe_row_index:
        link = links[i]
        print(link)
        driver.get(link)
        current_link = driver.current_url
        if link != current_link:
            print(i)
            print(f"Redirect occurred!")
            f.write(str(i) + "\n")
            f.write("Redirect occurred!" + "\n")
        else:
            wait = WebDriverWait(driver, 10)
            class_notes = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "des_txt")))
            print(i)
            print(class_notes.text)
            f.write(str(i) + "\n")
            f.write(class_notes.text + "\n")
    driver.quit()
    

# Năm sản xuất

In [ ]:
options = Options()
options.page_load_strategy = 'none'  # Set the page load strategy

miss_year_produce_row_index = [880, 977, 1784, 1806, 1948, 2540, 3543, 4054, 
    4180, 5585, 9913, 10518, 10703, 11321, 11561, 
    11573, 12886, 14640, 14703, 14916, 15334, 16888, 
    17358, 17839, 19102, 21962, 22456, 23327, 24057, 28767, 29309, 30526]
driver = webdriver.Chrome(options = options)  # Pass the options to the WebDriver
with open("year_produce.txt", "a", encoding='utf-8') as f:
    for i in miss_year_produce_row_index:
        link = links[i]
        print(link)
        driver.get(link)
        current_link = driver.current_url
        if link != current_link:
            print(i)
            print(f"Redirect occurred!")
            f.write(str(i) + "\n")
            f.write("Redirect occurred!" + "\n")
        else:
            wait = WebDriverWait(driver, 10)
            class_notes = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "inp")))
            print(i)
            print(class_notes.text)
            f.write(str(i) + "\n")
            f.write(class_notes.text + "\n")
    driver.quit()
    

# Drive type (Dẫn động)

In [ ]:
filt = (df["drive_type"] == "-")
df_none_drive_type = df[filt]
df_none_drive_type.head()

In [ ]:
miss_drive_type_links = df_none_drive_type["url"]

user_data_dir = "C:/Users/fpt47/AppData/Local/Google/Chrome/User Data - Copy"
service = Service(executable_path = "D:/fpt47/Downloads/chromedriver-win64/chromedriver.exe")

options = Options()
options.page_load_strategy = 'none'
options.add_argument(f"user-data-dir={user_data_dir}")
options.add_argument("--profile-directory=Default")

driver = webdriver.Chrome(service = service, options=options)
first_link_drive_type = r"https://bonbanh.com/xe-mercedes_benz-e_class-mercedes_e200-ex-2022-4879606"
driver.get(first_link_drive_type)

try:
    # Find the iframe
    wait = WebDriverWait(driver, 5)
    iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))

    # Switch to the iframe's context
    driver.switch_to.frame(iframe)

    # Now you can find the element within the iframe
    recaptcha_button = wait.until(EC.presence_of_element_located((By.ID, "recaptcha-anchor")))

    # Interact with the element (e.g., click it)
    time.sleep(random.uniform(9, 10))
    actions = ActionChains(driver)
    actions.move_to_element(recaptcha_button).perform()
    recaptcha_button.click()
except TimeoutException:
    # If iframe is not found, handle the exception
    print("The iframe is not present on the page. Performing alternative action...")
finally:
    # Switch back to the main content after interacting with the element
    driver.switch_to.default_content()

    miss_drive_type_links = df_none_drive_type["url"]

    with open("drive_type.txt", "a", encoding='utf-8') as f:
        start = 0
        for i, link in enumerate(miss_drive_type_links):
            driver.get(link)
            current_link = driver.current_url
            if link != current_link:
                print(i + 1 + start)
                print(f"Redirect occurred!")
                f.write("Redirect occurred!" + "\n")
            else:
                wait = WebDriverWait(driver, 3)
                class_notes = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[5]/div[1]/div[2]/div[6]/div[2]/span")))
                print(i + 1 + start)
                print(class_notes.text)
                f.write(class_notes.text + "\n")
        driver.quit()

In [57]:
found_drive_type = []
with open("drive_type.txt", "r", encoding = 'utf-8') as f:
    line = f.readline()
    while line:
        if len(line) > 3 and ('-' in line):
            found_drive_type.append(line[0:3])
        else:
            found_drive_type.append("-")
        line = f.readline()
df_none_drive_type["drive_type"] = found_drive_type
df.update(df_none_drive_type["drive_type"])

# Engine (Động cơ)

In [7]:
df_engine = pd.read_csv(r"D:\fpt47\Downloads\engine_car_detail_en.csv")

In [20]:
filt = (df_engine["engine"] == "-")
df_none_engine = df_engine[filt]
df_none_engine["engine_capacity"].head()
# df_none_engine.shape

4418     NaN
28992    NaN
29279    NaN
29375    NaN
29467    NaN
Name: engine_capacity, dtype: object

In [14]:
miss_engine_links = df_none_engine["url"]

user_data_dir = "C:/Users/fpt47/AppData/Local/Google/Chrome/User Data - Copy"
service = Service(executable_path = "D:/fpt47/Downloads/chromedriver-win64/chromedriver.exe")

options = Options()
options.page_load_strategy = 'none'
options.add_argument(f"user-data-dir={user_data_dir}")
options.add_argument("--profile-directory=Default")

driver = webdriver.Chrome(service = service, options=options)
first_link_engine = r"https://bonbanh.com/xe-volvo-xc60-recharge-2023-4686031"
driver.get(first_link_engine)

try:
    # Find the iframe
    wait = WebDriverWait(driver, 5)
    iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))

    # Switch to the iframe's context
    driver.switch_to.frame(iframe)

    # Now you can find the element within the iframe
    recaptcha_button = wait.until(EC.presence_of_element_located((By.ID, "recaptcha-anchor")))

    # Interact with the element (e.g., click it)
    time.sleep(random.uniform(4, 5))
    actions = ActionChains(driver)
    actions.move_to_element(recaptcha_button).perform()
    recaptcha_button.click()
except TimeoutException:
    # If iframe is not found, handle the exception
    print("The iframe is not present on the page. Performing alternative action...")
finally:
    # Switch back to the main content after interacting with the element
    driver.switch_to.default_content()

    miss_engine_links = df_none_engine["url"]

    with open("engine.txt", "a", encoding='utf-8') as f:
        start = 0
        for i, link in enumerate(miss_engine_links):
            driver.get(link)
            current_link = driver.current_url
            if link != current_link:
                print(i + 1 + start)
                print(f"Redirect occurred!")
                f.write("Redirect occurred!" + "\n")
            else:
                wait = WebDriverWait(driver, 15)
                class_notes = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[5]/div[1]/div[2]/div[1]/div[2]/span")))
                print(i + 1 + start)
                print(class_notes.text)
                f.write(class_notes.text + "\n")
        driver.quit()

1
Xăng 2.0 L
2
Xăng 2.4 L
3
Dầu 2.0 L
4
Xăng 1.6 L
5
Xăng 1.6 L
6
Xăng 1.6 L
7
Xăng 1.2 L
8
Xăng 1.5 L
9
Dầu 2.5 L
10
Xăng 3.0 L
11
Xăng 2.4 L
12
Dầu 2.5 L
13
Xăng 1.25 L
14
Xăng 1.4 L
15
Xăng 1.4 L
16
Xăng 1.5 L
17
Xăng 1.6 L
18
Xăng 1.2 L
19
Xăng 2.0 L
20
Xăng 1.0 L
21
Xăng 2.5 L
22
Xăng 2.0 L
23
Xăng 2.4 L
24
Xăng 2.0 L


In [ ]:
found_engine = []
found_engine_capacity = []
with open("engine.txt", "r", encoding = 'utf-8') as f:
    line = f.readline()
    while line:
        line = line.split()
        found_engine.append(line[0])
        found_engine_capacity.append(line[1] + " " + line[2])
        line = f.readline()
df_none_engine["engine"] = found_engine
df_none_engine["engine_capacity"] = found_engine_capacity
df_engine.update(df_none_engine[["engine", "engine_capacity"]])

In [ ]:
df_engine.head()

# Interior

1. Handle missing value

In [ ]:
df_v2 = pd.read_csv(r"D:\fpt47\Downloads\car_detail_en (version 2).csv")
df_v2.head()

In [50]:
df_v2.replace("-", np.nan, inplace = True)

In [51]:
def fill_group_mode(group):
    mode_series = group['interior_color'].mode()
    if not mode_series.empty:
        group['interior_color'] = group['interior_color'].fillna(mode_series.iloc[0])
    else:
        # Handle the case where mode_series is empty (no mode found)
        # For example, fill missing values with a default value
        group['interior_color'] = group['interior_color'].fillna('Default Value')
    return group

df_v2 = df_v2.groupby(["exterior_color", "grade", "year_of_manufacture"]).apply(fill_group_mode)

In [56]:
df1 = pd.read_csv("Dao_version_1.csv")
df1.set_index("ad_id", inplace = True)
# df_v2.set_index("ad_id", inplace = True)
df1.update(df_v2["interior_color"])

In [57]:
df1.to_csv("interior_color.csv")

In [58]:
df1.to_csv("Dao_version_1.csv")

In [47]:
df_interior_handling = df_v2[["ad_id", "exterior_color", "interior_color", "grade", "year_of_manufacture"]]
filt = (df_interior_handling["interior_color"].isna())
df_missing_interior = df_interior_handling.loc[filt]
df_missing_interior.head()
# print(df_missing_interior.shape)

,ad_id,exterior_color,interior_color,grade,year_of_manufacture
147,2676458,Red,NaN,CX5,2023.0
185,3157424,Green,NaN,2,2023.0
187,3164470,Green,NaN,CX5,2023.0
188,3164574,White,NaN,CX5,2023.0
207,3251335,Green,NaN,2,2023.0


In [ ]:
# Create a combined key column in both DataFrames
df_missing_interior.loc[:, 'key'] = df_missing_interior['exterior_color'] + '_' + df_missing_interior['grade'] + '_' + df_missing_interior['year_of_manufacture'].apply(str)
df_interior_handling.loc[:, 'key'] = df_interior_handling['exterior_color'] + '_' + df_interior_handling['grade'] + '_' + df_interior_handling['year_of_manufacture'].apply(str)
df_interior_handling.rename(columns = {"interior_color": "handling_interior_color"}, inplace = True)
df_missing_interior.rename(columns = {"interior_color": "missing_interior_color"}, inplace = True)
# df_interior_handling.head()
# df_missing_interior.head()
# Merge the DataFrames on the key column
merged_df = pd.merge(df_missing_interior, df_interior_handling[['key', "handling_interior_color"]], on='key', how='left')


In [ ]:
merged_df.loc[:, "interior_color"] = merged_df["handling_interior_color"]
merged_df.drop(["handling_interior_color", "missing_interior_color"], axis = 1, inplace = True)


In [ ]:
filt = (merged_df["interior_color"] == "-")
merged_df =  merged_df.loc[~filt]
merged_df.head(20)


# Found Mileage 

# Vanh

In [56]:
df_found_mileage = pd.read_csv(r"C:\Users\fpt47\OneDrive\Documents\found_mileage.csv")
import re
def text_to_number(text):
    """
    Converts text like "3 Billion 550 Million" to a numeric value.
    Handles cases with missing billions or millions.
    """
    word_to_num = {
        "Billion": 1,
        "Million": 1e-3
    }
    if pd.isna(text):
        return 0
    pattern = r"(\d+)?( Billion| Million)?"
    matches = re.findall(pattern, text)
    total = 0
    for match in matches:
        num, word = match
        if not word:
            pass
        else:
            total += float(num)*word_to_num[word.strip()]
    return round(total, 3)
df_found_mileage["Price"] = df_found_mileage["Price"].apply(text_to_number)
df_found_mileage["Mileage"] = df_found_mileage["Mileage"].apply(lambda x: int((str(x).split())[0].replace(".", "")))
df_found_mileage.to_csv("active_found_mileage.csv")


In [61]:
df_found_mileage["Compare_Mileage"] = df_found_mileage["Compare_Price"]*df_found_mileage["Mileage"]/df_found_mileage["Price"]
grouped_means = df_found_mileage.groupby("ID")["Compare_Mileage"].mean()
df_found_mileage = pd.DataFrame({"ID": grouped_means.index, "Mean_Mileage": grouped_means.values})

In [63]:
df_found_mileage["Mean_Mileage"] = df_found_mileage["Mean_Mileage"].apply(lambda x: int(x))

In [65]:
df_found_mileage.to_csv("active_found_mileage.csv")

# Tâm

In [20]:
df_new_mileage_Tam = pd.read_csv(r"D:\fpt47\Downloads\new_mileage_Tam.csv")
df_new_mileage_Tam.set_index("ad_id", inplace = True)
df = pd.read_csv("Dao_version_2.csv")
df.set_index("ad_id", inplace = True)
df["Tam_mileage"] = df_new_mileage_Tam["num_mileage"]

In [ ]:
df[df["Tam_mileage"] > 0]

In [23]:
df_filtered = df[(df["Tam_mileage"] > 0) & (df["mileage"] < 1000)]

In [ ]:
df_filtered

# Price

In [45]:
df = pd.read_csv("Dao_version_Vanh_mileage.csv")

In [ ]:
links = df["url"]
print(links)
links.isnull().sum()

In [ ]:
user_data_dir = "C:/Users/fpt47/AppData/Local/Google/Chrome/User Data - Copy"
service = Service(executable_path = "D:/fpt47/Downloads/chromedriver-win64/chromedriver.exe")

options = Options()
options.page_load_strategy = 'none'
options.add_argument(f"user-data-dir={user_data_dir}")
options.add_argument("--profile-directory=Default")

driver = webdriver.Chrome(service = service, options=options)
driver.get(links[0])

try:
    # Find the iframe
    wait = WebDriverWait(driver, 5)
    iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))

    # Switch to the iframe's context
    driver.switch_to.frame(iframe)

    # Now you can find the element within the iframe
    recaptcha_button = wait.until(EC.presence_of_element_located((By.ID, "recaptcha-anchor")))

    # Interact with the element (e.g., click it)
    time.sleep(random.uniform(4, 5))
    actions = ActionChains(driver)
    actions.move_to_element(recaptcha_button).perform()
    recaptcha_button.click()
except TimeoutException:
    # If iframe is not found, handle the exception
    print("The iframe is not present on the page. Performing alternative action...")
finally:
    # Switch back to the main content after interacting with the element
    driver.switch_to.default_content()

    with open("price.txt", "a", encoding = 'utf-8') as f:
        start = 921
        for i, link in enumerate(links[start:]):
            driver.get(link)
            current_link = driver.current_url
            if link != current_link:
                print(i + 1 + start)
                print(f"Redirect occurred!")
                f.write("Redirect occurred!" + "\n")
            else:
                wait = WebDriverWait(driver, 10)
                try:
                    class_notes = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[3]/h1")))
                    print(i + 1 + start)
                    print(class_notes.text)
                    f.write(class_notes.text + "\n")

                except TimeoutException:
                    print(i + 1 + start)
                    print("Timed out waiting for class notes element.")
                    f.write("Timed out waiting for class notes element." + "\n")
        driver.quit()

In [79]:
exist_price_index = []
with open("price.txt", "r", encoding = 'utf-8') as f:
    lines = f.readlines()
with open("price_v2.txt", "a", encoding = 'utf-8') as f:
    for i, line in enumerate(lines):
        if ("Million" in line) or ("Billion" in line):
            exist_price_index.append(i)
            price = line.split("-")[-1].strip()
            f.write(price + "\n")
        else:
            f.write("0" + "\n")
        
        

In [80]:
print(len(exist_price_index))

30568


In [85]:
df = pd.read_csv("Dao_version_price.csv")

In [82]:
with open("price_v2.txt", "r", encoding = 'utf-8') as f:
    lines = f.readlines()
for i in exist_price_index:
    df.loc[i, "price"] = lines[i]

In [86]:
df["price_in_billion"] = df["price"].apply(text_to_number)

In [87]:
df.to_csv("Dao_version_price_final.csv")

# Final

In [56]:
df.update(df_v2)


In [40]:
df = pd.read_csv("Dao_version_1.csv")

In [62]:
door_and_seat = pd.read_csv(r"D:\fpt47\Downloads\door_and_seat.csv")
car_detail_grade_processing = pd.read_csv(r"D:\fpt47\Downloads\car_detail_grade_processing.csv")
door_and_seat.set_index("ad_id", inplace = True)
car_detail_grade_processing.set_index("ad_id", inplace = True)
df.set_index("ad_id", inplace = True)

In [63]:
df.update(door_and_seat)
df.update(car_detail_grade_processing[["brand", "grade", "car_name"]])

In [29]:
df["engine_capacity"] = df_engine["engine_capacity"]
df.update(df_engine["engine"])

Số cột còn thiếu data: engine_capacity, fuel_consumption (An), condittion, car_model (Tâm)

In [4]:
engine_capacity_fuel_consumption = pd.read_csv(r"D:\fpt47\Downloads\An_engine_cap_fuel_consumption.csv")
engine_capacity_fuel_consumption.set_index("ad_id", inplace = True)
df.set_index("ad_id", inplace = True)

In [5]:
df.update(engine_capacity_fuel_consumption[["engine_capacity", "fuel_consumption"]])

In [10]:
new_mileage_Tam = pd.read_csv(r"D:\fpt47\Downloads\new_mileage_Tam.csv")
new_mileage_Tam.set_index("ad_id", inplace = True)
df.set_index("ad_id", inplace = True)
df.update(new_mileage_Tam[["mileage"]])

In [11]:
df.to_csv("Dao_version_1.csv", index = True)

In [42]:
#format mileage, engine_capacity
df["mileage"] = df["mileage"].apply(lambda x: int((str(x).split())[0].replace(",", "")))
df["engine_capacity"] = df["engine_capacity"].apply(lambda x: float((str(x).split())[0]))
df.to_csv("Dao_version_2.csv", index = True)

In [47]:
df = pd.read_csv("Dao_version_2.csv")
df_potential_error_mileage = df[(df["mileage"] < 100) & (df["mileage"] > 0)]
df_potential_error_mileage.to_csv("potential_error_mileage.csv")

Update mileage của Vanh

In [41]:
df_active_found_mileage = pd.read_csv("active_found_mileage.csv")
df_active_found_mileage.set_index("ID", inplace = True)
df = pd.read_csv("Dao_version_2.csv")
df.set_index("ad_id", inplace = True)


In [ ]:
df_active_found_mileage["Mean_Mileage"]

In [43]:
df.loc[df_active_found_mileage.index, "mileage"] = df_active_found_mileage["Mean_Mileage"]
df.to_csv("Dao_version_Vanh_mileage.csv")

In [3]:
df = pd.read_csv("Dao_version_price_final.csv")

In [8]:
len(df["grade"].unique())

622